In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
# Libraries..

import numpy as np
# from datetime import datetime
# from dateutil.relativedelta import relativedelta
# from itertools import product
#from tqdm import tqdm_notebook
import pandas as pd


# Plotly
import plotly.express as px
import plotly.graph_objects as go

# sklearn
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.impute import KNNImputer

#skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.utils import save_forecaster

# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.formula.api as smf

# pmdarima
from pmdarima.arima import auto_arima

#scipy
from scipy.stats import norm

#Neuralprophet
# from neuralprophet import NeuralProphet

# Modules from repo
from src.Utilities import utilities as tool

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.3f}'.format

c:\Users\JENMOSQU\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tensorflow WARNING From c:\Users\JENMOSQU\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



Using TensorFlow backend


In [4]:
#Import df to construct OLS...

df=pd.read_parquet('df.parquet')
print("columnas del df",df.columns)
df_regression=df.copy()
df_regression.dropna(inplace=True)
print(df_regression.columns)
df_regression.head(2)

columnas del df Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28', 'precipitacion', 'Temperatura', 'HRA2_MN_D', 'HRA2_MX_D', 'THSM_MEDIA_D', 'THSM_MX_D', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'VVAG_MEDIA_D', 'THSM_MN_D', 'PT_2_TT_D', 'PT_2_MX_D', 'PT_10_MX_D', 'PT_10_TT_D', 'TSSM_MEDIA_D', 'TSSM_MN_D', 'TSSM_MX_D', 'PTPG_TT_D', 'HR_CAL_MEDIA_D', 'BSHG_TT_D', 'TPR_CAL'], dtype='object')
Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'Demanda_S

,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,Demanda_SIN_1,Demanda_SIN_2,Demanda_SIN_3,Demanda_SIN_4,Demanda_SIN_5,Demanda_SIN_6,Demanda_SIN_7,Demanda_SIN_8,Demanda_SIN_9,Demanda_SIN_10,Demanda_SIN_11,Demanda_SIN_12,Demanda_SIN_13,Demanda_SIN_14,Demanda_SIN_15,Demanda_SIN_16,Demanda_SIN_17,Demanda_SIN_18,Demanda_SIN_19,Demanda_SIN_20,Demanda_SIN_21,Demanda_SIN_22,Demanda_SIN_23,Demanda_SIN_24,Demanda_SIN_25,Demanda_SIN_26,Demanda_SIN_27,Demanda_SIN_28,precipitacion,Temperatura,HRA2_MN_D,HRA2_MX_D,THSM_MEDIA_D,THSM_MX_D,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,VVAG_MEDIA_D,THSM_MN_D,PT_2_TT_D,PT_2_MX_D,PT_10_MX_D,PT_10_TT_D,TSSM_MEDIA_D,TSSM_MN_D,TSSM_MX_D,PTPG_TT_D,HR_CAL_MEDIA_D,BSHG_TT_D,TPR_CAL
28,2017-12-29,VIVDIC,180346.450,0,0,29,183383.600,183460.490,179631.410,149585.800,162494.320,180773.430,192100.300,194812.150,194268.750,194600.370,191040.380,166729.640,183056.120,193830.390,192875.800,190606.490,192531.990,190829.460,167216.620,179623.690,173629.320,195154.130,197097.780,196696.350,192717.880,167348.000,184393.750,193865.750,8.781,21.528,58.969,89.778,18.947,20.408,1.711,0.947,1.742,17.444,4.017,1.477,2.095,5.238,21.447,18.789,24.680,4.663,83.275,2.722,17.641
29,2017-12-30,SAVDIC,168819.760,0,0,30,180346.450,183383.600,183460.490,179631.410,149585.800,162494.320,180773.430,192100.300,194812.150,194268.750,194600.370,191040.380,166729.640,183056.120,193830.390,192875.800,190606.490,192531.990,190829.460,167216.620,179623.690,173629.320,195154.130,197097.780,196696.350,192717.880,167348.000,184393.750,7.821,21.566,57.029,90.712,19.178,20.653,1.688,0.865,1.502,17.725,2.818,1.435,2.055,4.677,21.523,18.695,24.915,3.344,82.926,3.924,17.702


In [6]:
#Gráfica de Demanda SIN Nueva...................................

trace1 = go.Scatter(
    x=df_regression['FECHA'],
    y=df_regression['Demanda_SIN'],
    #mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#276981'), 
    name='Demanda_SIN' 
)
fig = go.Figure(data=[trace1])
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)
fig.update_xaxes(title='Date-daily')
fig.update_yaxes(title='Electricity Demand SIN kWh')
fig.show()

In [40]:
import plotly.express as px

# Lista de categorías que deseas incluir en el gráfico
categorias_interes = ['LUNES', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO', 'DOMINGO', 'SAALF', 'MADLF', 'DOALF',
                      'LF']

# Filtrar el DataFrame para incluir solo las categorías de interés
df_filtrado = df[df['TIPO_DIA'].isin(categorias_interes)]

# Crear un gráfico de líneas interactivas
fig = px.line(df_filtrado, x='FECHA', y='Demanda_SIN', color='TIPO_DIA',
              title='Electricity Demand SIN kWh by Type of Day (Selected Categories)',
              labels={'Demanda_SIN': 'Electricity Demand SIN kWh', 'FECHA': 'Date', 'TIPO_DIA': 'Type of Day'},
              color_discrete_sequence=px.colors.qualitative.Set3)

# Mostrar el gráfico interactivo
fig.show()


In [41]:
import plotly.express as px

# List of categories you want to include in the plot
interest_categories = ['LUNES', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO', 'DOMINGO', 'SAALF', 'MADLF', 'DOALF', 'LF']

# Define the order of days of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'SAALF', 'MADLF', 'DOALF', 'LF']

# Filter the DataFrame to include only the categories of interest
filtered_df = df[df['TIPO_DIA'].isin(interest_categories)]

# Map custom names to the categories
custom_names = {'LUNES': 'Monday', 'MARTES': 'Tuesday', 'MIERCOLES': 'Wednesday', 'JUEVES': 'Thursday',
                'VIERNES': 'Friday', 'SABADO': 'Saturday', 'DOMINGO': 'Sunday', 'SAALF': 'SAALF', 'MADLF': 'MADLF',
                'DOALF': 'DOALF', 'LF': 'LF'}

# Create a new column with custom names
filtered_df['TIPO_DIA_Nombres'] = filtered_df['TIPO_DIA'].map(custom_names)

# List of custom colors (HEX codes)
custom_colors = ['#9e9764', '#642424', '#191919', '#497e76', '#b44c43', '#001a57', '#9e9764', '#642424', '#497e76', '#b44c43', '#d99058']

# Create an interactive line plot with the new column of custom names
fig = px.line(filtered_df, x='FECHA', y='Demanda_SIN', color='TIPO_DIA_Nombres',
              title='Electricity Demand SIN kWh by Type of Day (Selected Categories)',
              labels={'Demanda_SIN': 'Electricity Demand SIN kWh', 'FECHA': 'Date', 'TIPO_DIA_Nombres': 'Type of Day'},
              color_discrete_sequence=custom_colors,
              category_orders={'TIPO_DIA_Nombres': day_order})  # Set the order of the days of the week

# Show the interactive plot
fig.show()



In [39]:
import plotly.graph_objects as go

# Crear el gráfico de línea
trace1 = go.Scatter(
    x=df['FECHA'],
    y=df['Demanda_SIN'],
    mode='lines+markers',
    line=dict(shape='linear', color='#276981'),
    name='Demanda_SIN'
)

# Crear el rectángulo sombreado para el área de Covid
fig = go.Figure(data=[trace1])

# Filtrar el DataFrame para obtener solo las fechas con Covid (Covid=1)
df_covid = df[df['Covid'] == 1]

# Agregar el área sombreada para las fechas con Covid
fig.add_shape(
    go.layout.Shape(
        type="rect",
        x0=df_covid['FECHA'].min(),
        x1=df_covid['FECHA'].max(),
        y0=0,
        y1=max(df['Demanda_SIN']),
        fillcolor="rgba(34, 127, 134, 0.3)",  # Cambiar a #227f86
        layer="below",
        line=dict(color="rgba(255, 0, 0, 0)"),
    )
)

# Actualizar el diseño del gráfico
fig.update_layout(
    title="Demand SIN kWh indicating the Covid period",
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    xaxis_title='Date-daily',
    yaxis_title='Electricity Demand SIN kWh',
)

# Mostrar el gráfico interactivo
fig.show()

In [9]:
import plotly.graph_objects as go

# Crear el gráfico de línea
trace1 = go.Scatter(
    x=df['FECHA'],
    y=df['Demanda_SIN'],
    mode='lines+markers',
    line=dict(shape='linear', color='#276981'),
    name='Demanda_SIN'
)

# Crear el rectángulo sombreado para el área de Cambio_nivel
fig = go.Figure(data=[trace1])

# Filtrar el DataFrame para obtener solo las fechas con Cambio_nivel (Cambio_nivel=1)
df_cambio_nivel = df[df['Cambio_nivel'] == 1]

# Agregar el área sombreada para las fechas con Cambio_nivel
fig.add_shape(
    go.layout.Shape(
        type="rect",
        x0=df_cambio_nivel['FECHA'].min(),
        x1=df_cambio_nivel['FECHA'].max(),
        y0=0,
        y1=max(df['Demanda_SIN']),
        fillcolor="rgba(34, 127, 134, 0.3)",  # Cambiar a #227f86
        layer="below",
        line=dict(color="rgba(255, 0, 0, 0)"),
    )
)

# Actualizar el diseño del gráfico
fig.update_layout(
    title="Demand SIN kWh indicating the Period of Level Change",
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    xaxis_title='Date-daily',
    yaxis_title='Electricity Demand SIN kWh',
)

# Mostrar el gráfico interactivo
fig.show()


In [5]:
#Regresión Lineal: con todas las variables correlacionadas 

formula_regresion_lineal = f'Demanda_SIN ~ {" + ".join([f"Demanda_SIN_{i}" for i in range(1, 8)])} + \
    TIPO_DIA +Demanda_SIN_14  + Demanda_SIN_21 + Demanda_SIN_28 + \
          Covid + Cambio_nivel + Tiempo:Covid + Tiempo:Cambio_nivel + precipitacion + Temperatura + HRA2_MN_D + HRA2_MX_D + THSM_MEDIA_D + THSM_MX_D + VV_10_MEDIA_D + VV_10_VECT_MEDIA_D + \
            VVAG_MEDIA_D + THSM_MN_D + PT_2_TT_D + PT_2_MX_D + PT_10_MX_D +\
            PT_10_TT_D + TSSM_MEDIA_D + TSSM_MN_D + TSSM_MX_D + PTPG_TT_D + HR_CAL_MEDIA_D + BSHG_TT_D + TPR_CAL'\

modelo = smf.ols(formula=formula_regresion_lineal, data=df)
# Ajuste
resultado_modelo = modelo.fit()
print(resultado_modelo.summary())

# regresión por mínimos cuadrados ordinarios (OLS) que ha sido utilizada para modelar la relación entre la variable dependiente "Demanda_SIN" y varias variables independientes
# R-squared: El coeficiente de determinación (R-squared) es 0.974, lo que significa que aproximadamente el 97.4% de la variabilidad en la variable dependiente (Demanda_SIN) puede explicarse por las variables independientes incluidas en el modelo.
# F-statistic: El valor del estadístico F es 947.7, y el p-valor asociado es prácticamente cero (Prob (F-statistic): 0.00). Esto indica que al menos una de las variables independientes es significativa para predecir la variable dependiente.
# Muchos de los "TIPO_DIA" tienen p-valores cercanos a cero, lo que sugiere que son estadísticamente significativos en la predicción de la Demanda_SIN. Con al menos una categoría significativa en la variable "TIPO_DÍA", la variable ya es significativa para explicar la variabilidad del modelo
# Los rezagos estacionales, son significativos. (Demanda_SIN_7, Demanda_SIN_14, Demanda_SIN_21, Demanda_SIN_28)
# La variable Covid es significativa
# Interacción entre la tendencia y el cambio de nivel: el impacto de un cambio en "Tiempo"  o tendencia en la variable la demanda puede variar según el nivel de la variable "Cambio_nivel", y ésta es significativa.
# Las variables del clima significativas fueron: 
#THSM_MN_D: Temperatura húmeda mínima diaria
#PT_10_MX_D: Precipitación máxima diaria
#PT_10_TT_D: Precipitación total diaria
#TSSM_MN_D: Temperatura seca mínima diaria
#PTPG_TT_D: Precipitación total diaria
#HR_CAL_MEDIA_D: Humedad Relativa media diaria
#BSHG_TT_D: Brillo solar total diario
#VVAG_MEDIA_D: Velocidad del viento media diaria
#De lo que se infirió que como se esperaba la Temperatura y  la precipitación debía ser una de las variables que más explicará la variación de la demanda de energía. 
#De igual manera también resulto significativa la variable Humedad Relativa media diaria y Brillo solar total diario y la Velocidad del viento media diaria.



                            OLS Regression Results                            
Dep. Variable:            Demanda_SIN   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     1048.
Date:                Mon, 27 Nov 2023   Prob (F-statistic):               0.00
Time:                        08:00:32   Log-Likelihood:                -18113.
No. Observations:                1979   AIC:                         3.638e+04
Df Residuals:                    1902   BIC:                         3.681e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              9240.53

In [11]:
#Regresión Lineal: sin la variable tipo día,  y sin las variables que no son significativas del clima

formula_regresion_lineal = f'Demanda_SIN ~ {" + ".join([f"Demanda_SIN_{i}" for i in range(1, 8)])} + \
          Demanda_SIN_14  + Demanda_SIN_21 + Demanda_SIN_28 + \
          Covid + Tiempo:Cambio_nivel + VV_10_MEDIA_D +  PT_10_TT_D + THSM_MEDIA_D' 

modelo = smf.ols(formula=formula_regresion_lineal, data=df)
# Ajuste
resultado_modelo = modelo.fit()
print(resultado_modelo.summary())

                            OLS Regression Results                            
Dep. Variable:            Demanda_SIN   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     481.2
Date:                Sun, 26 Nov 2023   Prob (F-statistic):               0.00
Time:                        16:27:25   Log-Likelihood:                -21009.
No. Observations:                2041   AIC:                         4.205e+04
Df Residuals:                    2025   BIC:                         4.214e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -1.049e+04   8

In [ ]:
df.columns 

#OLS significativas:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

#THSM_MN_D: Temperatura húmeda mínima diaria
#PT_10_MX_D: Precipitación máxima diaria
#PT_10_TT_D: Precipitación total diaria
#TSSM_MN_D: Temperatura seca mínima diaria
#PTPG_TT_D: Precipitación total diaria
#HR_CAL_MEDIA_D: Humedad Relativa media diaria
#BSHG_TT_D: Brillo solar total diario
#VVAG_MEDIA_D: Velocidad del viento media diaria


#Seleccionadas::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

# VV_10_MEDIA_D: Velocidad del viento media diaria.
# VV_10_VECT_MEDIA_D: Persistencia diaria.
# PT_10_TT_D: Precipitación total diaria.
# THSM_MEDIA_D: Temperatura húmeda máxima diaria